In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('C:/Users/tomek/Desktop/CME_metadata.csv', parse_dates=['from_date', 'to_date'])

In [7]:
from bundles.expiration_downloader import ExpirationDownloader

In [8]:
a = ExpirationDownloader(df)

KeyboardInterrupt: 

In [51]:
df[df['code'] == 'JYZ2018']

,code,name,description,refreshed_at,from_date,to_date
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)","For contract details, please see <a href=http:...",2018-11-01 22:11:15,2014-02-14,2018-11-01


In [21]:
df.drop(df[
    df['description'].str.contains('Dataset description', regex=False) == True
    ].index, inplace=True)

In [22]:
df[df['code'] == 'JYZ2018']

,code,name,description,refreshed_at,from_date,to_date
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)","For contract details, please see <a href=http:...",2018-11-01 22:11:15,2014-02-14,2018-11-01


In [23]:
# drop various indexes included in Quandl file
df.drop(df[df['code'].str.contains('INDEX', regex=False) == True].index, inplace=True)

In [24]:
df[df['code'] == 'JYZ2018']

,code,name,description,refreshed_at,from_date,to_date
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)","For contract details, please see <a href=http:...",2018-11-01 22:11:15,2014-02-14,2018-11-01


In [25]:
# fix an error in urls in quandl meta data file:
df['description'] = df['description'].str.replace('/mac-swap-futures/', '/swap-futures/')
df['year'] = df.code.apply(lambda x: x[-4:])
df['root_symbol'] = df.code.apply(lambda x: x[:-5])
df.rename(columns={'code': 'symbol'}, inplace=True)
df['exch_symbol'] = df['symbol'].apply(lambda x: x[:-4] + x[-2:])

In [26]:
df[df['symbol'] == 'JYZ2018']

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)","For contract details, please see <a href=http:...",2018-11-01 22:11:15,2014-02-14,2018-11-01,2018,JY,JYZ18


In [27]:
# extract url from description field
df['description'] = df['description'].apply(
lambda x: x.split('<a href=')[-1].split('>http')[0].strip())

# change the url to get calendar data instead of contract specs
df['description'] = df['description'].str.replace(
    'contract_specifications', 'product_calendar_futures')

In [28]:
df[df['symbol'] == 'JYZ2018']

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)",http://www.cmegroup.com/trading/fx/g10/japanes...,2018-11-01 22:11:15,2014-02-14,2018-11-01,2018,JY,JYZ18


In [29]:
df.count()

symbol          36042
name            36042
description     36042
refreshed_at    36042
from_date       36042
to_date         36042
year            36042
root_symbol     36042
exch_symbol     36042
dtype: int64

In [30]:
df[df['symbol'] == 'JYZ2018']

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)",http://www.cmegroup.com/trading/fx/g10/japanes...,2018-11-01 22:11:15,2014-02-14,2018-11-01,2018,JY,JYZ18


In [31]:
# Filter out non-active contracts (they don't need updating)
cutoff_date = df['to_date'].max() - pd.Timedelta(days=2)

In [32]:
cutoff_date

Timestamp('2018-10-30 00:00:00')

In [33]:
df = df[df['to_date'] >= cutoff_date]

In [34]:
df.count()

symbol          6807
name            6807
description     6807
refreshed_at    6807
from_date       6807
to_date         6807
year            6807
root_symbol     6807
exch_symbol     6807
dtype: int64

In [42]:
for row in df.iteritems():
    print(row)

('symbol', 4          0DF2019
5          0DF2020
12         0DG2019
13         0DG2020
20         0DH2019
21         0DH2020
28         0DJ2019
29         0DJ2020
36         0DK2019
37         0DK2020
44         0DM2019
45         0DM2020
52         0DN2019
53         0DN2020
60         0DQ2019
61         0DQ2020
68         0DU2019
69         0DU2020
75         0DV2018
76         0DV2019
77         0DV2020
83         0DX2018
84         0DX2019
85         0DX2020
91         0DZ2018
92         0DZ2019
93         0DZ2020
100        0FF2020
109        0FG2020
118        0FH2020
            ...   
164577     Z9V2021
164578     Z9V2022
164583     Z9X2018
164584     Z9X2019
164585     Z9X2020
164586     Z9X2021
164587     Z9X2022
164592     Z9Z2018
164593     Z9Z2019
164594     Z9Z2020
164595     Z9Z2021
164596     Z9Z2022
164636      ZH2019
164637      ZH2020
164675      ZM2019
164679    ZNCF2019
164683    ZNCG2019
164687    ZNCH2019
164691    ZNCJ2019
164695    ZNCK2019
164699    ZNCM2019
1

In [17]:
df = df[df['year'] > '2018']

In [18]:
df.count()

symbol          6246
name            6246
description     6246
refreshed_at    6246
from_date       6246
to_date         6246
year            6246
root_symbol     6246
exch_symbol     6246
dtype: int64

In [16]:
df[df['symbol'] == 'JYZ2018']

,symbol,name,description,refreshed_at,from_date,to_date,root_symbol,exch_symbol
76502,JYZ2018,"Japanese Yen Futures, December 2018 (JYZ2018)",http://www.cmegroup.com/trading/fx/g10/japanes...,2018-11-01 22:11:15,2014-02-14,2018-11-01,JY,JYZ18


In [19]:
len(df['root_symbol'].unique())

256

In [45]:
df[df['year'] < '2018']

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol


In [16]:
df.head()

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol
4,0DF2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-11-01 22:11:08,2015-02-12,2018-11-01,2019,0D,0DF19
5,0DF2020,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-11-01 22:11:08,2017-12-04,2018-11-01,2020,0D,0DF20
12,0DG2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-11-01 22:11:08,2015-02-12,2018-11-01,2019,0D,0DG19
13,0DG2020,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-11-01 22:11:08,2017-12-04,2018-11-01,2020,0D,0DG20
20,0DH2019,Mini European 3.5% Fuel Oil Barges FOB Rdam (P...,http://www.cmegroup.com/trading/energy/refined...,2018-11-01 22:11:08,2015-02-12,2018-11-01,2019,0D,0DH19


In [17]:
df[df['symbol'] == 'JYZ2018']

,symbol,name,description,refreshed_at,from_date,to_date,year,root_symbol,exch_symbol
